In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("operations.csv")

In [3]:
data.head()

,date_operation,libelle,montant,solde_avt_ope,categ
0,2023-03-31,DON XX XX XX XX XX XX XX,-1.44,1515.25,AUTRE
1,2023-04-03,CARTE XX XX RAPT XX,-24.00,1513.81,TRANSPORT
2,2023-04-03,CARTE XX XX RAPT XX,-73.00,1489.81,TRANSPORT
3,2023-04-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,1416.81,AUTRE
4,2023-04-03,VIREMENT XX XX XX XX XX XX,4.80,2092.81,AUTRE


In [4]:
data.dtypes

date_operation     object
libelle            object
montant           float64
solde_avt_ope     float64
categ              object
dtype: object

In [7]:
data["date_operation"] = pd.to_datetime(data["date_operation"]) # On formate la date.
data.head()

,date_operation,libelle,montant,solde_avt_ope,categ
0,2023-03-31,DON XX XX XX XX XX XX XX,-1.44,1515.25,AUTRE
1,2023-04-03,CARTE XX XX RAPT XX,-24.00,1513.81,TRANSPORT
2,2023-04-03,CARTE XX XX RAPT XX,-73.00,1489.81,TRANSPORT
3,2023-04-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,1416.81,AUTRE
4,2023-04-03,VIREMENT XX XX XX XX XX XX,4.80,2092.81,AUTRE


In [9]:
data.isnull().sum()

date_operation    0
libelle           0
montant           2
solde_avt_ope     0
categ             1
dtype: int64

In [10]:
# On voit que, pour deux transactions, le montant est manquant. Regardons ça de plus près.
data.loc[data["montant"].isnull(), :]

,date_operation,libelle,montant,solde_avt_ope,categ
107,2023-06-12,CARTE XX XX LES ANCIENS ROBINSON XX,NaN,4667.19,COURSES
269,2023-09-11,CARTE XX XX XX XX,NaN,3401.93,AUTRE


In [18]:
# Comme ces deux opérations ne se succèdent pas, on peut en fait remédier au problème :
# montant = solde_avt_ope suivante -  solde_avt_ope courante
for i in data.loc[data["montant"].isnull(), :].index:
    data.iloc[i, 2] = data.iloc[i+1, 3] - data.iloc[i, 3]

In [20]:
data.iloc[[107, 108, 269, 270], :]

,date_operation,libelle,montant,solde_avt_ope,categ
107,2023-06-12,CARTE XX XX LES ANCIENS ROBINSON XX,-26.58,4667.19,COURSES
108,2023-06-12,CARTE XX XX CHEZ LUC XX,-9.20,4640.61,RESTAURANT
269,2023-09-11,CARTE XX XX XX XX,-12.00,3401.93,AUTRE
270,2023-09-11,CARTE XX XX XX XX XX XX,-1.44,3389.93,AUTRE


In [21]:
data.isnull().sum()

date_operation    0
libelle           0
montant           0
solde_avt_ope     0
categ             1
dtype: int64

In [22]:
# Occupons-nous de la dernière valeur manquante
data.loc[data["categ"].isnull(), :]

,date_operation,libelle,montant,solde_avt_ope,categ
156,2023-07-06,PRELEVEMENT XX TELEPHONE XX XX,-36.48,3295.68,NaN


In [31]:
# On voit que le libellé est assez explicite. Voyons s'il n'y a pas d'autres opérations avec le même libellé
data.loc[data["libelle"] == data.loc[data["categ"].isnull(), "libelle"].values[0], :]

,date_operation,libelle,montant,solde_avt_ope,categ
8,2023-04-05,PRELEVEMENT XX TELEPHONE XX XX,-7.02,2056.02,FACTURE TELEPHONE
62,2023-05-09,PRELEVEMENT XX TELEPHONE XX XX,-7.02,4090.10,FACTURE TELEPHONE
102,2023-06-07,PRELEVEMENT XX TELEPHONE XX XX,-6.38,4688.91,FACTURE TELEPHONE
156,2023-07-06,PRELEVEMENT XX TELEPHONE XX XX,-36.48,3295.68,NaN
204,2023-08-07,PRELEVEMENT XX TELEPHONE XX XX,-7.46,3751.73,FACTURE TELEPHONE
260,2023-09-05,PRELEVEMENT XX TELEPHONE XX XX,-6.38,3453.96,FACTURE TELEPHONE
308,2023-10-06,PRELEVEMENT XX TELEPHONE XX XX,-13.58,2413.58,FACTURE TELEPHONE


In [33]:
# Tout de suite, la réponse nous saute aux yeux : il faut choisir la catégorie "FACTURE TELEPHONE".
data.iloc[156, -1] = "FACTURE TELEPHONE"

In [34]:
# On s'est occupé de toutes les valeurs manquantes. Regardons maintenant s'il y a des doublons.
# Les variables qui ne devraient pas avoir de doublons sont "libelle", "montant", "solde_avt_ope", "categ".
data.loc[data[["libelle", "montant", "solde_avt_ope", "categ"]].duplicated(keep=False), :]

,date_operation,libelle,montant,solde_avt_ope,categ
43,2023-04-25,CARTE XX XX LES ANCIENS ROBINSON XX,-32.67,3647.67,COURSES
44,2023-04-25,CARTE XX XX LES ANCIENS ROBINSON XX,-32.67,3647.67,COURSES


In [35]:
data.drop_duplicates(subset = ["libelle", "montant", "solde_avt_ope", "categ"], inplace=True, ignore_index=True)

In [36]:
data.iloc[44, :]

date_operation        2023-04-26 00:00:00
libelle           CARTE XX XX XX XX XX XX
montant                             -10.0
solde_avt_ope                      3615.0
categ                               AUTRE
Name: 44, dtype: object

In [37]:
# Passons maintenant à la détection des outliers.
data.describe()

,montant,solde_avt_ope
count,308.000000,308.000000
mean,-45.782013,3395.301071
std,872.818105,667.109412
min,-15000.000000,1416.810000
25%,-20.447500,3010.737500
50%,-9.600000,3452.465000
75%,-2.715000,3787.232500
max,1071.600000,4709.310000


In [39]:
# On voit que le min des montants est -15000, ce qui est étonnant dans la mesure où l'amplitude des solde_avt_ope est de 3000...
data.loc[data["montant"]<-10000,:]

,date_operation,libelle,montant,solde_avt_ope,categ
198,2023-08-03,CARTE XX XX XX XX,-15000.0,3797.35,AUTRE


In [41]:
#Regardons cette transaction de plus près
data.iloc[197:200, :]

,date_operation,libelle,montant,solde_avt_ope,categ
197,2023-08-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,3121.35,AUTRE
198,2023-08-03,CARTE XX XX XX XX,-15000.00,3797.35,AUTRE
199,2023-08-03,CARTE XX XX L'EPICERIE DEMBAS XX XX,-10.51,3782.96,AUTRE


In [42]:
# On voit très clairement qu'il y a une erreur. Le montant devrait être de -14.39.
data.iloc[198, 2] = -14.39
# Et voilà c'est fini !